In [11]:
!pip install simpy

### Goal
- feature out Simpy events, process and timeout

most basic simulation

In [12]:
from simpy import Environment

In [13]:
sim_duration = 10
env = Environment()
print(f'start at {env.now}')
env.run(until=sim_duration)
print(f'stop at {env.now}')

start at 0
stop at 10


you need a process
in this process customer waits for 2 

In [14]:
from simpy import Environment, Process
class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now:2.1f}") 
        duration = len(self.name)
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now:2.1f}") 

In [15]:
sim_duration = 10
env = Environment()
Visitor(env, "Alice")
print(f'start at {env.now}')
env.run(until=sim_duration)
print(f'stop at {env.now}')

start at 0
Alice: Here I am at 0.0
Alice: I must leave at 5.0
stop at 10


please note that visitor whow once and leave

events.Event
|
+— events.Timeout
|
+— events.Initialize
|
+— events.Process
|
+— events.Condition
|  |
|  +— events.AllOf
|  |
|  +— events.AnyOf
.
.

generator is an iterator, but it easier to write (example with __iter__ and __next__)
as it is an iterator ou may user next


In [16]:
sim_duration = 10
env = Environment()
visitor = Visitor(env, "Alice")
try:
    timeout = next(visitor.visit())
    print(timeout)
    print(f"timeout {timeout}")
    print(f"now {timeout.env.now}")
    print(f"timeout value {timeout.value}")
    print(f"triggered {timeout.triggered}")
    print(f"processed {timeout.processed}")
except StopIteration:
    print("Unexpected error:", sys.exc_info()[0])

Alice: Here I am at 0.0
<Timeout(5) object at 0x1082dc820>
timeout <Timeout(5) object at 0x1082dc820>
now 0
timeout value None
triggered True
processed False


now add a callback
never -processed but automaticaly triggered

run sim until event occur

In [17]:
def my_callback(timeout):
    print(f'Called back from {timeout}')
    print(timeout)
    print(f"now {timeout.env.now}")
    print(f"timeout {timeout}")
    print(f"timeout value {timeout.value}")
    print(f"triggered {timeout.triggered}")
    print(f"processed {timeout.processed}")

env = Environment()
visited = Visitor(env, "Alice")
visited.callbacks.append(my_callback)
env.run(until=visited)
print(f'stop at {env.now}')

Alice: Here I am at 0.0
Alice: I must leave at 5.0
Called back from <Visitor(visit) object at 0x1082dca30>
<Visitor(visit) object at 0x1082dca30>
now 5
timeout <Visitor(visit) object at 0x1082dca30>
timeout value None
triggered True
processed True
stop at 5


until n stands for until timeout(n)
stops when the process is done

In [18]:
env = Environment()
visited = Visitor(env, "Alice")
print(visited)
print(f'start at {env.now}')
env.run(until=visited)
print(f'stop at {env.now}')

<Visitor(visit) object at 0x1082dc760>
start at 0
Alice: Here I am at 0.0
Alice: I must leave at 5.0
stop at 5


In [19]:
env = Environment()
alice_visited = Visitor(env, "Alice")
bob_visited = Visitor(env, "Bob")
print(f'start at {env.now}')
env.run(until= alice_visited | bob_visited)
print(f'stop at {env.now}')

start at 0
Alice: Here I am at 0.0
Bob: Here I am at 0.0
Bob: I must leave at 3.0
stop at 3


In [20]:
env = Environment()
alice_visited = Visitor(env, "Alice")
bob_visited = Visitor(env, "Bob")
print(f'start at {env.now}')
env.run(until= alice_visited & bob_visited)
print(f'stop at {env.now}')

start at 0
Alice: Here I am at 0.0
Bob: Here I am at 0.0
Bob: I must leave at 3.0
Alice: I must leave at 5.0
stop at 5


until expect any event or condition

run sim until event occur
event.succeed
happy with visits

condition and multiple condition